In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir gdrive
!google-drive-ocamlfuse gdrive
!ls gdrive

In [0]:
file= open('poemShakespeare.txt').read()

In [0]:
raw_text= file.lower()

In [0]:
tok= raw_text.split()

In [0]:
text=""
for i in tok:
  text+= i+ " "

In [20]:
text

"a lover's complaint from off a hill whose concave womb reworded a plaintful story from a sistering vale, my spirits to attend this double voice accorded, and down i laid to list the sad-tuned tale; ere long espied a fickle maid full pale, 5 tearing of papers, breaking rings a-twain, storming her world with sorrow's wind and rain. upon her head a platted hive of straw, which fortified her visage from the sun, whereon the thought might think sometime it saw 10 the carcass of beauty spent and done: time had not scythed all that youth begun, nor youth all quit; but, spite of heaven's fell rage, some beauty peep'd through lattice of sear'd age. oft did she heave her napkin to her eyne, 15 which on it had conceited characters, laundering the silken figures in the brine that season'd woe had pelleted in tears, and often reading what contents it bears; as often shrieking undistinguish'd woe, 20 in clamours of all size, both high and low. sometimes her levell'd eyes their carriage ride, as the

In [0]:
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
seq_length=10
vocab= len(chars)

In [28]:
len(text)

14544

In [0]:
sequences=[]
y=[]
for i in range(len(text)):
  if i+seq_length>=len(text):
    break
  sequences.append(text[i:i+seq_length])
  y.append(text[i+seq_length])
  

In [45]:
len(y)

14534

In [0]:
X=[]
for seq in sequences:
  tmp=[]
  for i in seq:
    tmp.append(char_to_int[i])
  X.append(tmp)
  
Y=[]
for i in y:
  Y.append(char_to_int[i])

In [47]:
X[:6]

[[19, 0, 30, 33, 40, 23, 36, 2, 37, 0],
 [0, 30, 33, 40, 23, 36, 2, 37, 0, 21],
 [30, 33, 40, 23, 36, 2, 37, 0, 21, 33],
 [33, 40, 23, 36, 2, 37, 0, 21, 33, 31],
 [40, 23, 36, 2, 37, 0, 21, 33, 31, 34],
 [23, 36, 2, 37, 0, 21, 33, 31, 34, 30]]

In [0]:
from keras.utils import np_utils
datax = np_utils.to_categorical(X)
datay = np_utils.to_categorical(Y)

In [49]:
len(datax[0][0])

45

In [53]:
datay.shape

(14534, 45)

In [55]:
datax.shape

14534

In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint

In [56]:
model = Sequential()
model.add(LSTM(256, input_shape=(datax.shape[1], datax.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(datay.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(datax, datay, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
14534/14534 [==============================] - 15s 1ms/step - loss: 3.0202

Epoch 00001: loss improved from inf to 3.02016, saving model to weights.hdf5
Epoch 2/50
14534/14534 [==============================] - 13s 874us/step - loss: 2.6179

Epoch 00002: loss improved from 3.02016 to 2.61789, saving model to weights.hdf5
Epoch 3/50
14534/14534 [==============================] - 13s 875us/step - loss: 2.3671

Epoch 00003: loss improved from 2.61789 to 2.36705, saving model to weights.hdf5
Epoch 4/50
14534/14534 [==============================] - 13s 874us/step - loss: 2.2422

Epoch 00004: loss improved from 2.36705 to 2.24218, saving model to weights.hdf5
Epoch 5/50
14534/14534 [==============================] - 13s 866us/step - loss: 2.1549

Epoch 00005: loss improved from 2.24218 to 2.15488, saving model to weights.hdf5
Epoch 6/50
14534/14534 [==============================] - 13s 867us/step - loss: 2.0660

Epoch 00006: loss improved from 2.15488 to 2.06598, saving model to

In [73]:
tstr="give me a "
print(tstr,end='')
for i in range(1000):
  test=[]
  for i in tstr:
    test.append(char_to_int[i])

  test= np_utils.to_categorical(test,num_classes=vocab)
  test= test.reshape(1,test.shape[0],test.shape[1])
  ans= model.predict(test)
  pp=chars[numpy.argmax(ans)]
  print(pp,end='')
  tstr+=pp
  tstr=tstr[1:]
  
  

give me a several fair, their kind acceptance weepingly beseech'd, with the inundation of the wind upon his chin; his phoenix down began but to appear like unshorn velvet on that termless skin whose bare out-bragg'd the web it seem'd to wear: 95 yet show'd his visage by that cost more dear; and nice affections woeld veir a sare the fromived harm was sent me from a nun, or sister sanctified, of holiest note; which late her noble suit in court did shun, 235 whose rarest havings made the blossoms dote; for she was sought by spirits to attend this double voice accorded, and down i laid to list the sad-tuned tale; ere long espied a fickle maid full pale, 5 tearing of papers, breaking rings a-twain, storming her world with sorrow's wind and rain. upon her head a platted hive of straw, which fortified her visage from the sun, whereon the thought might think sometime diverted their poor balls are tied to the orbed earth; sometimes they do extend 25 their view right on; anon their gazes lend to

's'